# Reforestaction
https://www.reforestaction.com/en/projects

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
with open('../input/Reforestaction/data.json', 'r') as file:
    data = json.load(file)
type(data)

In [ ]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
df.info()

### Fix geometries

In [ ]:
df['geometry'] = df.apply(lambda row: Point(row['attributes.centroidLongitude'], row['attributes.centroidLatitude']), axis=1)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

### Harmonize nomenclature

In [ ]:
gdf = gdf[['id', 'attributes.externalId', 'geometry']]

columns_rename_mapping = {
    'id': 'project_id_reported',
    'attributes.externalId':'site_id_reported'
}
gdf.rename(columns=columns_rename_mapping, inplace=True)

In [ ]:
gdf['host_name'] = 'reforestaction'
gdf['url'] = 'https://www.reforestaction.com/en/projects'

gdf = gdf.assign(project_description_reported=None,site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,country=None)
gdf.info()

### Save it

In [ ]:
gdf.to_file('../midsave/reforestaction.gpkg')